# Отток клиентов

# Превью

Для работы представлена таблица, содержащая категориальные и количественные признаки (часть из них бинарные). В целях подготовки признаков будет необходимо произвести отбор наиболее информативных из них (влияющих на принятие решения). Также преобразовать категориальные данные в количественные метдом прямого кодирования, избежав при этом dammy-ловушки. Масштабирование признаков будет необходимо провести для равномерности их влияния на принятие решения, т.к. ,например, есть значения уровня дохода и количества зарегистрираванных жилых помещений, которые отличаются на 5-6 порядков.В дальнейшем необходимо разделить выборки на валидационную, тестовую и тренировачную, предварительно выделив отдельно целевой признак, оценить в нем соотношение классов и принять решение о необходимости введения весового коэффицента либо применения процедур up- downsampling. Следующим шагом будет определение оптимальной модели и ее параметров. Находить ее будем на основе значения метрики f1-score. Также сравним значения метрики AUC-ROC.   

# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [2]:
users_bh =pd.read_csv('/datasets/Churn.csv')
users_bh.info()
display(users_bh.head())
users_bh = users_bh.fillna(0)
users_bh = users_bh.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1 )
part_1 = users_bh[users_bh['Exited']== 1]['Exited'].count()/users_bh.shape[0]
part_0 = users_bh[users_bh['Exited']== 0]['Exited'].count()/users_bh.shape[0]
print('Часть положительного класса =', part_1)
print('Часть отрицательного класса =', part_0)
users_bh_ohe = pd.get_dummies(users_bh, drop_first = True)
users_bh_ohe.info()
target = users_bh_ohe['Exited']
features = users_bh_ohe.drop(['Exited'], axis=1)
names = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features[names])
features[names] = scaler.transform(features[names])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Часть положительного класса = 0.2037
Часть отрицательного класса = 0.7963
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          10000 non-null int64
Age                  10000 non-null int64
Tenure               10000 non-null float64
Balance              10000 non-null float64
NumOfProducts        10000 non-null int64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Geography_Germany    10000 non-null uint8
Geography_Spain      10000 non-null uint8
Gender_Male          10000 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


# Выводы по пункту 1

При подготовке данных в датасет загружены данные из файла, а также проведен анализ отсутствующих значений и типов данных. В столбце "Tenure" около 10% отстутствующих значений. Спишем это на невнимательное заполнение поля о количестве недвижимости у клиента. Скорее всего если этот пункт не заполнен, то недвижимости у клиента просто нет. Так что заполним пропуски нулями. Проведя анализ, выявили 3 неинформативных поля: RowNumber, CustomerId, Surname. Они применяются скорее для индексации данных и самой простой привязки к клиенту, но не влияют на уход клиента из банка. Просто удалим эти столбцы. В оставшихся данных есть 2 столбца с категориальным типом данных. Применим для них отображение. В результате определения распределения классов удалось выяснить, что зачений положительного класса примерно в 5 раз меньше, чем отрицательного. Придется применять весовые коэффиценты, а если не поможет, то и upsampling. Также применили масштабирование для количественных параметров (за исключением параметров с бинарными значениями).   


# 2. Исследование задачи

In [4]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, target,
                                                                                        test_size = 0.4, random_state = 12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test,target_valid_test,
                                                                            test_size = 0.5, random_state = 12345)


model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
predict_test = model.predict(features_test)
print('LogisticRegression f1_score valid =', f1_score(target_valid, predict_valid))
print('LogisticRegression f1_score test =', f1_score(target_test, predict_test))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression auc_roc valid=', roc_auc_score(target_valid, probabilities_one_valid))
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('LogisticRegression auc_roc test=', roc_auc_score(target_test, probabilities_one_test))

best_score = 0
best_depth = 0
for depth in range(1, 21, 1):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    score = f1_score(target_valid, predict_valid)
    if  score > best_score:
        best_score = score  
        best_depth = depth
print('best_score =', best_score)
print('best_depth =', best_depth)
best_estimators = 0
for estimators in range(5, 101, 5):    
    model = RandomForestClassifier(n_estimators=estimators, 
                                  max_depth = best_depth, random_state=12345)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    score = f1_score(target_valid, predict_valid)
    if  score > best_score:
        best_score = score  
        best_estimators = estimators
print('best_score_1 =', best_score)
print('best_estimators =', best_estimators)
model = RandomForestClassifier(n_estimators= best_estimators, 
                                  max_depth = best_depth, random_state=12345)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
predict_test =  model.predict(features_test)
print('RandomForestClassifier_best f1_score valid =', f1_score(target_valid, predict_valid))
print('RandomForestClassifier_best f1_score test =', f1_score(target_test, predict_test))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('RandomForestClassifier auc_roc valid=', roc_auc_score(target_valid, probabilities_one_valid))
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('RandomForestClassifier auc_roc test=', roc_auc_score(target_test, probabilities_one_test))

LogisticRegression f1_score valid = 0.33389544688026984
LogisticRegression f1_score test = 0.2743055555555555
LogisticRegression auc_roc valid= 0.7586378456196807
LogisticRegression auc_roc test= 0.7386500087696812


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


best_score = 0.5907046476761619
best_depth = 12
best_score_1 = 0.592814371257485
best_estimators = 40
RandomForestClassifier_best f1_score valid = 0.592814371257485
RandomForestClassifier_best f1_score test = 0.5238828967642527
RandomForestClassifier auc_roc valid= 0.843305215976385
RandomForestClassifier auc_roc test= 0.8544112995468248


# Выводы по пункту 2 

В данном пункте изначально разбил данные на выборки (тренировочную, валидационную и тестовую) в сооотношении 3:1:1. Обучил модели логистической регрессии и случайного леса без учета дисбаланса классов. Для модели случайного леса определил параметры, с которыми значение метрики f1_score будет максимальным. В итоге модель логистической регрессии показала очень слабый результат даже для валидационной выборки, а модель случайного леса с количеством наблюдателей 40 и глубиной дерева 12 для валидационной выборки преодолела порог f1_score (0.592814371257485), а для тестовой выборки нет (0.5238828967642527). Придется учитывать баланс классов для улучшения качества модели. Количество наблюдателей для случайного леса изменял в диапазоне от 5 до 100 с шагом 5, а глубину дерева от 1 до 20.    

# 3. Борьба с дисбалансом

## class_weight = 'balanced'

In [5]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear', class_weight = 'balanced')
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
predict_test = model.predict(features_test)
print('LogisticRegression f1_score valid =', f1_score(target_valid, predict_valid))
print('LogisticRegression f1_score test =', f1_score(target_test, predict_test))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression auc_roc valid=', roc_auc_score(target_valid, probabilities_one_valid))
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('LogisticRegression auc_roc test=', roc_auc_score(target_test, probabilities_one_test))



best_score = 0
best_depth = 0
for depth in range(1, 21, 1):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, class_weight = 'balanced',  random_state=12345)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    score = f1_score(target_valid, predict_valid)
    if  score > best_score:
        best_score = score  
        best_depth = depth
print('best_score =', best_score)
print('best_depth =', best_depth)
best_estimators = 0
for estimators in range(5, 101, 5):    
    model = RandomForestClassifier(n_estimators=estimators, 
                                  max_depth = best_depth,  class_weight = 'balanced' ,random_state=12345)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    score = f1_score(target_valid, predict_valid)
    if  score > best_score:
        best_score = score  
        best_estimators = estimators
print('best_score =', best_score)
print('best_estimators =', best_estimators)
model = RandomForestClassifier(n_estimators= best_estimators, 
                                  max_depth = best_depth, class_weight = 'balanced', random_state=12345)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
predict_test =  model.predict(features_test)
print('RandomForestClassifier_best f1_score valid =', f1_score(target_valid, predict_valid))
print('RandomForestClassifier_best f1_score test =', f1_score(target_test, predict_test))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('RandomForestClassifier auc_roc valid=', roc_auc_score(target_valid, probabilities_one_valid))
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('RandomForestClassifier auc_roc test=', roc_auc_score(target_test, probabilities_one_test))

LogisticRegression f1_score valid = 0.4888888888888888
LogisticRegression f1_score test = 0.4797238999137188
LogisticRegression auc_roc valid= 0.7635782940859793
LogisticRegression auc_roc test= 0.7417846076354692
best_score = 0.6196891191709843
best_depth = 6
best_score = 0.6241134751773049
best_estimators = 95
RandomForestClassifier_best f1_score valid = 0.6241134751773049
RandomForestClassifier_best f1_score test = 0.597585513078471
RandomForestClassifier auc_roc valid= 0.8537494177922684
RandomForestClassifier auc_roc test= 0.8508584543474383


##  upsampling

In [6]:
def upsample(features, target, repeat):
    features_zeros = features_train[target == 0]
    features_ones = features_train[target == 1]
    target_zeros = target_train[target == 0]
    target_ones = target_train[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled
print('объемы тренировочных выборок до upsampling`a', features_train.shape[0], 'объектов', target_train.shape[0], 'объектов')
part_11 = target_train[target_train== 1].count()/target_train.shape[0]
part_00 = target_train[target_train== 0].count()/target_train.shape[0]
print('Часть положительного класса =', part_11)
print('Часть отрицательного класса =', part_00)
features_train, target_train = upsample(features_train, target_train, 2)
print('объемы тренировочных выборок после upsampling`a', features_train.shape[0], 'объектов', target_train.shape[0], 'объектов')
part_11 = target_train[target_train== 1].count()/target_train.shape[0]
part_00 = target_train[target_train== 0].count()/target_train.shape[0]
print('Часть положительного класса =', part_11)
print('Часть отрицательного класса =', part_00)

model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
predict_test = model.predict(features_test)
print('LogisticRegression f1_score valid =', f1_score(target_valid, predict_valid))
print('LogisticRegression f1_score test =', f1_score(target_test, predict_test))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression auc_roc valid=', roc_auc_score(target_valid, probabilities_one_valid))
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('LogisticRegression auc_roc test=', roc_auc_score(target_test, probabilities_one_test))



best_score = 0
best_depth = 0
for depth in range(1, 21, 1):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth,  random_state=12345)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    score = f1_score(target_valid, predict_valid)
    if  score > best_score:
        best_score = score  
        best_depth = depth
print('best_score =', best_score)
print('best_depth =', best_depth)
best_estimators = 0
for estimators in range(5, 101, 5):    
    model = RandomForestClassifier(n_estimators=estimators, 
                                  max_depth = best_depth, random_state=12345)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    score = f1_score(target_valid, predict_valid)
    if  score > best_score:
        best_score = score  
        best_estimators = estimators
        if best_estimators == 0:
            best_estimators = 1            
print('best_score =', best_score)
print('best_estimators =', best_estimators)
model = RandomForestClassifier(n_estimators= best_estimators, 
                                  max_depth = best_depth, random_state=12345)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
predict_test =  model.predict(features_test)
print('RandomForestClassifier_best f1_score valid =', f1_score(target_valid, predict_valid))
print('RandomForestClassifier_best f1_score test =', f1_score(target_test, predict_test))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('RandomForestClassifier auc_roc valid=', roc_auc_score(target_valid, probabilities_one_valid))
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('RandomForestClassifier auc_roc test=', roc_auc_score(target_test, probabilities_one_test))

объемы тренировочных выборок до upsampling`a 6000 объектов 6000 объектов
Часть положительного класса = 0.19933333333333333
Часть отрицательного класса = 0.8006666666666666
объемы тренировочных выборок после upsampling`a 7196 объектов 7196 объектов
Часть положительного класса = 0.33240689271817675
Часть отрицательного класса = 0.6675931072818232
LogisticRegression f1_score valid = 0.4672435105067985
LogisticRegression f1_score test = 0.4395061728395062
LogisticRegression auc_roc valid= 0.7612207308294873
LogisticRegression auc_roc test= 0.7403214950132743
best_score = 0.6196808510638299
best_depth = 8
best_score = 0.6205059920106524
best_estimators = 55
RandomForestClassifier_best f1_score valid = 0.6205059920106524
RandomForestClassifier_best f1_score test = 0.5922974767596282
RandomForestClassifier auc_roc valid= 0.8548684664194679
RandomForestClassifier auc_roc test= 0.857445459328917


# Выводы по пункту 3

При учете веса класса значения метрики f1_score увеличились. Особенно это заметно для модели логистичесткой регрессии (0.33389544688026984 и 0.48889). Однако лучше показатели у модели случайного леса при значениях параметров n_estimators = 95 и max_depth = 6 (0.6241134751773049 для валидационной выборки и 0.597585513078471 для тестовой, что больше порогового значения 0.59).
Во второй части задания применил метод upsampling и увеличил количество значений положительного класса тренировочной выборки в 2 раза. соотношение классов в было ~1/4, а стало ~1/2. Итогом стало также увеличение метрики f1_score для всех моделей, а лучшая из случайного леса снова преодолела порог качества модели при параметрах best_depth = 8 и best_estimators = 55 (0.5922974767596282), но оказалась хуже, чем для баланса классов (0.597585513078471). А вот метрика AUC-ROC у последней модели хоть не значительно, но выше, что для валидационной, что для тестовой выборки.  

# 4. Тестирование модели

Проведено в ходе двух предыдущих пунктов.

# Вывод по проекту

В ходе выполнения проекта проводилась работа с данными, направленная на предсказание ухода клиента из банка. Для этого была дана таблица с характеристиками клиентов. Предварительный анализ датасета помог определить направления для подготовки данных 
перед подачей на вход моделей предсказания. Таким образом были удалены неинформативные поля, заполнены пропуски данных, проведены процедуры прямого кодирования и масштабирования признаков. Далее проведено выделение целевого признака и деление выборок на тренировочную, тестовую и валидационную. В целях тестирования выбраны модели логистической регрессии и случайного леса для задачи классификации. В ходе проведенного обучения и тестов модель логистической регрессии показала себя заметно хуже, чем модель случайного леса, опираясь на значение метрики f1_score. Особо ощутима была разница без учета весов классов. Что касается применения методов учета и подгонки весов классов, то максимальное значение метрики f1_score получено для модели случайного леса со сбалансированными классами при параметрах n_estimators = 95 и max_depth = 6 (0.597585513078471). А вот с наилучшим значением метрики AUC-ROC (0.857445459328917) получилась модель случайного леса с балансом классов 1 к 2.

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [X]  Весь код выполняется без ошибок
- [X]  Ячейки с кодом расположены в порядке исполнения
- [X]  Выполнен шаг 1: данные подготовлены
- [X]  Выполнен шаг 2: задача исследована
    - [X]  Исследован баланс классов
    - [X]  Изучены модели без учёта дисбаланса
    - [X]  Написаны выводы по результатам исследования
- [X]  Выполнен шаг 3: учтён дисбаланс
    - [X]  Применено несколько способов борьбы с дисбалансом
    - [X]  Написаны выводы по результатам исследования
- [X]  Выполнен шаг 4: проведено тестирование
- [X]  Удалось достичь *F1*-меры не менее 0.59
- [X]  Исследована метрика *AUC-ROC*